## Networks

One of the reasons Docker containers and services are so powerful is that you can connect them together, or connect them to non-Docker workloads. We have 4 basic types of networks:

Here is the summary.

* **bridge**

* **host**

* **none**

* **overlay**

There are plugins to extend these possibilities.

## Bridge

The default network driver. If you don’t specify a driver, this is the type of network you are creating. Bridge networks are usually used when your applications run in standalone containers that need to communicate. All containers without a `--network` specified, are attached to the default bridge network. On a user-defined bridge network, containers can resolve each other by name or alias.

Let's launch a nginx container on the network by default.

In [ ]:
%%bash
docker container run -p 80:80 -d --name nginx --rm nginx:latest;

Its default ip address on the network is

In [ ]:
%%bash
docker inspect nginx | grep IPAddress;
"SecondaryIPAddresses": null,
            "IPAddress": "172.17.0.2",
                    "IPAddress": "172.17.0.2",

We can reach for it via another container on the same network by default.

In [ ]:
%%bash
docker container run --rm curlimages/curl curl 172.17.0.2;
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   612  100   612    0     0   597k      0 --:--:-- --:--:-- --:--:--  597k
<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>

Change the nginx network container to be isolated. We will create a network only for this container.

In [ ]:
%%bash
docker network create nginxnet;
docker rm -f nginx;
docker container run -p 80:80 -d --name nginx --rm --network nginxnet nginx:latest;
docker inspect nginx | grep IPAddress;
"SecondaryIPAddresses": null,
            "IPAddress": "",
                    "IPAddress": "172.18.0.2",

1. Network creation

2. Delete the nginx container and recreate it on the new network

3. Its new ip address on the network is 172.18.0.2

Can we now still wait for it?

In [ ]:
%%bash
docker container run --rm curlimages/curl curl 172.18.0.2;
docker container run --rm curlimages/curl curl 172.18.0.2;
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:16 --:--:--     0

Of course not. They are no longer part of the same network. We curl from the default network while the nginx container is on the nginxnet network.

## None

If you want to completely disable the networking stack on a container, you can use the `--network none` flag when starting the container.

## Host

For standalone containers, remove network isolation between the container and the Docker host, and use the host’s networking directly. `host` is only available for swarm services on Docker 17.06 and higher. We will see briefly docker swarm in another reading

## Overlay

Overlay networks connect multiple Docker daemons together and enable swarm services to communicate with each other. You can also use overlay networks to facilitate communication between a swarm service and a standalone container, or between two standalone containers on different Docker daemons. This strategy removes the need to do OS-level routing between these containers